In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from eval_utils import load_model
path_name = '/u/scr/smurty/LanguageExplanations/trained_models/t5-large-sst-no-exp'
model_obj = load_model(path_name, primary_mode='exp_applies_predictor')

Some weights of T5ForConditionalGenerationMultipleHeads were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


primary mode: exp_applies_predictor
splicing parts from pretrained model


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from collections import Counter
from eval_utils import predict_stuff
import itertools
import numpy as np


def single_prompt_pred(model, data, prompt, examine=False):
    prompted_inps = [(prompt, ex) for ex in data[0]]
    probs = predict_stuff(prompted_inps, itertools.repeat(0), 
                          model, 'p1', verbose=False, mode='task_predictor')
    #print(np.mean(probs.argmax(axis=1)==data[1]))
    return probs.argmax(axis=1)

def get_prompting_scores(model_obj, data, prompt_set, examine=False):
    orig_preds = single_prompt_pred(model_obj, data, '')
    all_preds = []
    for prompt in patches:
        all_preds.append(single_prompt_pred(model_obj, (inputs, labels), prompt))
    p = np.stack(all_preds, axis=1)
    model_preds = []
    for x in p:
        counts = Counter(x)
        model_preds.append(counts[1] > counts[0])
    return np.array(model_preds), orig_preds

In [ ]:
from data_fns import knowledge_absn
inputs, labels, patches = knowledge_absn(abstraction=True)

model_preds, orig_preds = get_prompting_scores(model_obj, (inputs, labels), patches)
print(np.mean(model_preds == orig_preds))

In [ ]:
from data_fns import knowledge_checklists
out, patches = knowledge_checklists(abstraction=True)
inputs = []
labels = []
for key in out:
    inputs += out[key]['instances']
    labels += out[key]['labels']

In [ ]:
model_preds, orig_preds = get_prompting_scores(model_obj, (inputs, labels), patches)
print(np.mean(model_preds == labels))
print(np.mean(orig_preds == labels))